In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from time import sleep
from random import randint
import pandas as pd
import json

In [ ]:
# attention to make the api available on other computers
# jupyter kernelgateway --generate-config
# /Users/laetitiahoquetis/.jupyter/jupyter_kernel_gateway_config.py
# If you want your api to be accessible from other computers, edit the ~/.jupyter/jupyter_kernel_gateway_config.py file. Replace
# # c.KernelGatewayApp.ip = '127.0.0.1'
# with
# c.KernelGatewayApp.ip = '*'

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [ ]:
# url = 'https://www.gamespot.com/games/metroid-prime-4/'
# result = requests.get(url, headers=headers)
# soup = BeautifulSoup(result.content, "html.parser")
# #print(page.content.decode())

In [ ]:
# same for https://www.gameinformer.com/2023
# inside div then <span class="calendar_entry">
# get name for s href link
# <a href="/product/a-space-for-the-unbound" hreflang="en">A Space for the Unbound</a>
# (<em>PlayStation 5, Xbox Series X/S, PlayStation 4, Xbox One, Switch, PC</em>)		– <time datetime="00Z" class="datetime">January 19</time>
#  </span>
# then platform m
# <em>PlayStation 5, Xbox Series X/S, PlayStation 4, Xbox One, Switch, PC</em>
# then date 
# <time datetime="00Z" class="datetime">January 19</time>

In [ ]:
pages = ['', 'xbox-one', 'ps4', '3ds', 'pc', 'nintendo-switch']
event = []
date = []

In [ ]:
for page in pages: 
    url = f"https://www.gamespot.com/gamespot-50/{page}"
    print(url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    titles = soup.find_all("h3")
    for t in titles:
        try:
            event.append(t.a.text.strip())
        except Exception:
            event.append('n/a')
    dates = soup.find_all(class_="media-release-date")
    for d in dates:
        date.append(d.findChild().text)
    print("page parsed")
    sleep(randint(2,20))
    

In [ ]:
# titles = soup.find_all("h3")
# for t in titles:
#     try:
#         event.append(t.a.text.strip())
#     except Exception:
#         event.append('n/a')
    
# dates = soup.find_all(class_="media-release-date")
# for d in dates:
#     date.append(d.findChild().text)

In [ ]:
pd.set_option('display.max_rows', 500)
df = pd.DataFrame(list(zip(event,date)),
               columns =['event', 'date'])
df = df.drop_duplicates(subset=['event'])
df = df[df['event']!= "n/a"]
df

In [ ]:
df['paged'] = df['event'].astype(str)
df['paged'] = df['paged'].str.lower()
df['paged'] = df['paged'].str.replace(":","").str.replace("'","").str.replace(".","").str.replace("&","and").str.replace("- ","")
df['paged'] = df['paged'].str.replace(" ","-")  
df['paged'] = df['paged'].apply(lambda x: x.replace(',', ''))
df['paged'] = df['paged'].apply(lambda x: x.replace('!', ''))

In [ ]:
# df.event needs to be modified into paged list where space replaced by "-"
# also lower cased and remove when any ": or ' or . or - " or ','
# then serach for platform = []
# then search for des = []

In [ ]:
# # #url = f'https://www.gamespot.com/games/'{df.event}
# url = 'https://www.gamespot.com/games/resident-evil-4/'
# result = requests.get(url, headers=headers)
# soup = BeautifulSoup(result.content, "html.parser")
# #print(result.content.decode())
# #ul class = game-module__platform
# # #p class = game-module__description

In [ ]:
# res = soup.find('ul', attrs={'class' : 'game-module__platform'}).get_text()
# type(res)

In [ ]:
# soup.find('p', attrs={'class' : 'game-module__description'}).get_text()

In [ ]:
platform = []
des = []

In [ ]:
df

In [ ]:
for page in df.paged: 
    url = f"https://www.gamespot.com/games/{page}"
    print(url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    res = soup.find('ul', attrs={'class' : 'game-module__platform'}).get_text()
    platform.append(res)
    desc = soup.find('p', attrs={'class' : 'game-module__description'}).get_text()
    des.append(desc)
    print("page parsed")
    sleep(randint(2,10))
    

In [ ]:
df.head(3)

In [ ]:
df['platform'] = platform
df['des'] = des

In [ ]:
df.sample(5)

In [ ]:
# add another website screening
other = []
# #url = f'https://www.gamespot.com/games/'{df.event}
url = 'https://www.digitaltrends.com/gaming/upcoming-ps5-games//'
result = requests.get(url, headers=headers)
soup = BeautifulSoup(result.content, "html.parser")
#print(result.content.decode())
#div class = "b-media__title b-media__title--small"
result = soup.find_all("div", {"class":"b-media__title b-media__title--small"})
for res in result : 
    other.append(res.text.strip())


In [ ]:
df1 = pd.DataFrame(list(other),
               columns =['other'])
df1['other1'] = df1['other'].str.split(' ').str.get(0)

In [ ]:
test = []
for x in df1['other1']:
    if df['event'].str.contains(x).any():
        test.append(None)
    else:
        test.append(x)
df1['test'] = test
df2 = df1.dropna()
df2['date'] = df2['other'].str.split(' - ').str.get(-1)
df2['event'] = df2['other'].str.split(' - ').str.get(0)
df2 = df2[df2['date'].str.contains(", 2023")]
df2 = df2[['event','date']]

In [ ]:
df3 = df.append(df2)
df3

In [ ]:
df3.to_csv('events2023.csv')

In [ ]:
# for page in df.paged[:3]: 
#     url = f"https://www.gamespot.com/games/{page}"
#     print(url)
#     response = requests.get(url, headers=headers)
#     soup = BeautifulSoup(response.content, "html.parser")
#     plat = soup.find_all('ul',class_="game-module__platform")
#     for p in plat:
#         pi = p.text
#         platform.append(pi.split()[0])
#     desc = soup.find_all(class_="game-module__description")
#     for d in desc:
#         des.append(d.text)
#     print("page parsed")
#     sleep(randint(2,20))
    